In [1]:
#!/usr/bin/env python
from glob import glob
from astroquery.mast import Mast, Observations
import os
import shutil
import numpy as np
import json
import requests
import asdf
from astropy import log
from astropy.io import ascii, fits
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch
import matplotlib.pyplot as plt
import matplotlib as mpl

# do this before importing webb
os.environ["CRDS_PATH"] = "/orange/adamginsburg/jwst/brick/crds/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

from jwst.pipeline import calwebb_image3
from jwst.pipeline import Detector1Pipeline, Image2Pipeline

# Individual steps that make up calwebb_image3
from jwst.tweakreg import TweakRegStep
from jwst.skymatch import SkyMatchStep
from jwst.outlier_detection import OutlierDetectionStep
from jwst.resample import ResampleStep
from jwst.source_catalog import SourceCatalogStep
from jwst import datamodels
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

from align_to_catalogs import realign_to_vvv, merge_a_plus_b, retrieve_vvv
from saturated_star_finding import iteratively_remove_saturated_stars, remove_saturated_stars

from destreak import destreak

import crds

import datetime
import jwst

def print(*args, **kwargs):
    now = datetime.datetime.now().isoformat()
    from builtins import print as printfunc
    return printfunc(f"{now}:", *args, **kwargs)


print(jwst.__version__)

# see 'destreak410.ipynb' for tests of this
medfilt_size = {'F410M': 15, 'F405N': 256, 'F466N': 55}

#basepath = '/orange/adamginsburg/jwst/brick/'

filtername = 'F466N'
module = 'merged'
field = '002'
regionname = 'cloudc'

2023-06-02 13:37:10,686 - stpipe - WARNING - /blue/adamginsburg/adamginsburg/jwst/cloudc/reduction/brick-jwst-2221/reduction/filtering.py:20: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm

fatal: detected dubious ownership in repository at '/blue/adamginsburg/adamginsburg/repos/dask'
To add an exception for this directory, call:

	git config --global --add safe.directory /blue/adamginsburg/adamginsburg/repos/dask


2023-06-02T13:37:17.782816: 1.9.0


In [2]:
calwebb_image3.Image3Pipeline.call??

Signature: calwebb_image3.Image3Pipeline.call(*args, **kwargs)
Source:   
    @classmethod
    def call(cls, *args, **kwargs):
        """
        Creates and runs a new instance of the class.

        Gets a config file from CRDS if one is available

        To set configuration parameters, pass a `config_file` path or
        keyword arguments.  Keyword arguments override those in the
        specified `config_file`.

        Any positional `*args` will be passed along to the step's
        `process` method.

        Note: this method creates a new instance of `Step` with the given
        `config_file` if supplied, plus any extra `*args` and `**kwargs`.
        If you create an instance of a Step, set parameters, and then use
        this `call()` method, it will ignore previously-set parameters, as
        it creates a new instance of the class with only the `config_file`,
        `*args` and `**kwargs` passed to the `call()` method.

        If not used with a `config_file` or spe

In [4]:
vvvcat = fits.open('/orange/adamginsburg/jwst/cloudc//F466N/pipeline/jw02221-o001_t001_nircam_clear-F466N-merged_vvvcat.fits')

In [9]:
vvvcat[1].data

FITS_rec([(266.602823, -28.60222 , 'VVV J174624.67-283607.99', -1, 18.806, 0.084,  0, 17.736, 0.053,  0, 16.528, 0.034, 16,    nan,   nan,  0,    nan,   nan,  0, 266.602823, -28.60222 ),
          (266.606742, -28.598289, 'VVV J174625.61-283553.84', -1, 16.278, 0.011,  0, 15.811, 0.011,  0, 15.225, 0.011, 16, 14.721, 0.027, 16, 14.421, 0.058, 16, 266.606742, -28.598289),
          (266.613329, -28.597491, 'VVV J174627.19-283550.96', -1, 17.865, 0.037, 16, 17.298, 0.036, 16, 16.693, 0.039, 16, 16.261, 0.111, 16,    nan,   nan,  0, 266.613329, -28.597491),
          ...,
          (266.59499 , -28.520139, 'VVV J174622.79-283112.49',  1,    nan,   nan,  0,    nan,   nan,  0, 17.2  , 0.061, 16, 14.584, 0.024, 16,    nan,   nan,  0, 266.59499 , -28.520139),
          (266.592745, -28.520205, 'VVV J174622.25-283112.73', -1,    nan,   nan,  0,    nan,   nan,  0,    nan,   nan,  0,    nan,   nan,  0, 14.23 , 0.049, 16, 266.592745, -28.520205),
          (266.593806, -28.520737, 'VVV J174622.51

In [12]:
from astropy.table import Table

In [16]:

vvvdr2fn = '/orange/adamginsburg/jwst/cloudc//F466N/pipeline/jw02221-o002_t001_nircam_clear-F466N-merged_vvvcat.ecsv'
#(f'{basepath}/{filtername.upper()}/pipeline/jw02221-o{field}_t001_nircam_clear-{filtername}-{module}_vvvcat.ecsv')

os.path.exists(vvvdr2fn)

False

In [13]:
Table(vvvcat[1].data)

RAJ2000,DEJ2000,iauname,mClass,Zmag3,e_Zmag3,Zperrbits,Ymag3,e_Ymag3,Yperrbits,Jmag3,e_Jmag3,Jperrbits,Hmag3,e_Hmag3,Hperrbits,Ksmag3,e_Ksmag3,Ksperrbits,RA,DEC
float64,float64,str24,int16,float32,float64,int32,float64,float64,int32,float64,float64,int32,float64,float64,int32,float64,float64,int32,float64,float64
266.602823,-28.60222,VVV J174624.67-283607.99,-1,18.806,0.084,0,17.736,0.053,0,16.528,0.034,16,nan,nan,0,nan,nan,0,266.602823,-28.60222
266.606742,-28.598289,VVV J174625.61-283553.84,-1,16.278,0.011,0,15.811,0.011,0,15.225,0.011,16,14.721,0.027,16,14.421,0.058,16,266.606742,-28.598289
266.613329,-28.597491,VVV J174627.19-283550.96,-1,17.865,0.037,16,17.298,0.036,16,16.693,0.039,16,16.261,0.111,16,nan,nan,0,266.613329,-28.597491
266.611708,-28.596065,VVV J174626.81-283545.83,1,19.567,0.167,0,19.022,0.17,0,17.988,0.126,0,15.987,0.086,16,14.496,0.062,16,266.611708,-28.596065
266.603496,-28.60398,VVV J174624.83-283614.32,-1,16.636,0.014,0,16.304,0.016,0,15.796,0.018,0,14.848,0.03,16,13.669,0.029,16,266.603496,-28.60398
266.610196,-28.601597,VVV J174626.44-283605.75,-1,18.836,0.087,0,18.411,0.097,0,17.722,0.099,0,17.499,0.345,0,nan,nan,0,266.610196,-28.601597
266.613424,-28.595982,VVV J174627.22-283545.53,1,19.395,0.143,0,18.878,0.149,0,17.766,0.103,16,nan,nan,0,nan,nan,0,266.613424,-28.595982
266.603983,-28.60527,VVV J174624.95-283618.97,1,nan,nan,0,nan,nan,0,15.802,0.018,0,13.3,0.008,16,11.588,0.004,16,266.603983,-28.60527
266.614028,-28.605808,VVV J174627.36-283620.90,-1,18.678,0.075,0,17.789,0.056,0,16.494,0.033,16,14.852,0.03,16,nan,nan,0,266.614028,-28.605808


In [2]:
#def main(filtername, module, Observations=None, regionname='brick', field='001'):
log.info(f"Processing filter {filtername} module {module}")
# sanity check
if regionname == 'brick':
    assert field == '001'
elif regionname == 'cloudc':
    assert field == '002'
basepath = f'/orange/adamginsburg/jwst/{regionname}/'
os.environ["CRDS_PATH"] = f"{basepath}/crds/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
# Files created in this notebook will be saved
# in a subdirectory of the base directory called `Stage3`
output_dir = f'/orange/adamginsburg/jwst/{regionname}/{filtername}/pipeline/'
print('Output directory set as:', output_dir)
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
os.chdir(output_dir)
# the files are one directory up
for fn in glob("../*cal.fits"):
    try:
        os.link(fn, './'+os.path.basename(fn))
    except Exception as ex:
        print(f'Failed to link {fn} to {os.path.basename(fn)} because of {ex}')


2023-05-31 11:53:55,827 - stpipe - INFO - Processing filter F466N module merged


INFO: Processing filter F466N module merged [unknown]
2023-05-31T11:53:55.829992: Output directory set as: /orange/adamginsburg/jwst/cloudc/F466N/pipeline/


In [3]:
Observations.cache_location = output_dir
obs_table = Observations.query_criteria(
                                        proposal_id="2221",
                                        proposal_pi="Ginsburg*",
                                        #calib_level=3,
                                        )
print("Obs table length:", len(obs_table))
msk = ((np.char.find(obs_table['filters'], filtername.upper()) >= 0) |
       (np.char.find(obs_table['obs_id'], filtername.lower()) >= 0))
data_products_by_obs = Observations.get_product_list(obs_table[msk])
print("data prodcts by obs length: ", len(data_products_by_obs))
products_asn = Observations.filter_products(data_products_by_obs, extension="json")
print("products_asn length:", len(products_asn))
#valid_obsids = products_asn['obs_id'][np.char.find(np.unique(products_asn['obs_id']), 'jw02221-o001', ) == 0]
#match = [x for x in valid_obsids if filtername.lower() in x][0]
asn_mast_data = products_asn#[products_asn['obs_id'] == match]
print("asn_mast_data:", asn_mast_data)
manifest = Observations.download_products(asn_mast_data, download_dir=output_dir)
print("manifest:", manifest)

2023-05-31T11:54:18.309766: Obs table length: 15
2023-05-31T11:54:21.907023: data prodcts by obs length:  29612
2023-05-31T11:54:22.384921: products_asn length: 34
2023-05-31T11:54:22.385046: asn_mast_data:   obsID   obs_collection dataproduct_type                obs_id                ...  size parent_obsid    dataRights    calib_level
--------- -------------- ---------------- ------------------------------------ ... ----- ------------ ---------------- -----------
 90220722           JWST            image jw02221-o001_t001_nircam_f444w-f466n ... 14244     90220722 EXCLUSIVE_ACCESS           3
127345709           JWST            image jw02221-o002_t001_nircam_f444w-f466n ... 10148    127345709 EXCLUSIVE_ACCESS           3
127235693           JWST            image   jw02221002001_04201_00001_nrcalong ...  1795    127345709 EXCLUSIVE_ACCESS           2
127235308           JWST            image   jw02221002001_04201_00001_nrcblong ...  1795    127345709 EXCLUSIVE_ACCESS           2
1272356

In [ ]:
# MAST creates deep directory structures we don't want
for row in manifest:
    try:
        shutil.move(row['Local Path'], os.path.join(output_dir, os.path.basename(row['Local Path'])))
    except Exception as ex:
        print(f"Failed to move file with error {ex}")


In [12]:

products_fits = Observations.filter_products(data_products_by_obs, extension="fits")
print("products_fits length:", len(products_fits))
uncal_mask = np.array([uri.endswith('_uncal.fits') and f'jw02221{field}' in uri for uri in products_fits['dataURI']])
uncal_mask &= products_fits['productType'] == 'SCIENCE'
print("uncal length:", (uncal_mask.sum()))

already_downloaded = np.array([os.path.exists(os.path.basename(uri)) for uri in products_fits['dataURI']])
uncal_mask &= ~already_downloaded
print(f"uncal to download: {uncal_mask.sum()}; {already_downloaded.sum()} were already downloaded")

2023-05-31T12:44:07.783626: products_fits length: 29060
2023-05-31T12:44:08.267028: uncal length: 32
2023-05-31T12:44:11.483467: uncal to download: 16; 112 were already downloaded


In [7]:
field

'002'

In [11]:
products_fits[(products_fits['productType'] == 'SCIENCE') & np.array([uri.endswith('_uncal.fits') for uri in products_fits['dataURI']])]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str9,str4,str5,str36,str64,str1,str69,str9,str28,str11,str1,str7,str5,str4,str51,int64,str9,str16,int64
90107202,JWST,image,jw02221001001_05101_00001_nrcalong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_05101_00001_nrcalong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_05101_00001_nrcalong_uncal.fits,25223040,90220722,EXCLUSIVE_ACCESS,1
90112172,JWST,image,jw02221001001_05101_00001_nrcblong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_05101_00001_nrcblong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_05101_00001_nrcblong_uncal.fits,25223040,90220722,EXCLUSIVE_ACCESS,1
90104132,JWST,image,jw02221001001_05101_00002_nrcalong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_05101_00002_nrcalong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_05101_00002_nrcalong_uncal.fits,25223040,90220722,EXCLUSIVE_ACCESS,1
90110302,JWST,image,jw02221001001_05101_00002_nrcblong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_05101_00002_nrcblong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_05101_00002_nrcblong_uncal.fits,25223040,90220722,EXCLUSIVE_ACCESS,1
90108068,JWST,image,jw02221001001_05101_00003_nrcalong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_05101_00003_nrcalong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_05101_00003_nrcalong_uncal.fits,25223040,90220722,EXCLUSIVE_ACCESS,1
90111220,JWST,image,jw02221001001_05101_00003_nrcblong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_05101_00003_nrcblong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_05101_00003_nrcblong_uncal.fits,25223040,90220722,EXCLUSIVE_ACCESS,1
90107199,JWST,image,jw02221001001_05101_00004_nrcalong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_05101_00004_nrcalong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_05101_00004_nrcalong_uncal.fits,25223040,90220722,EXCLUSIVE_ACCESS,1
90113041,JWST,image,jw02221001001_05101_00004_nrcblong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_05101_00004_nrcblong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_05101_00004_nrcblong_uncal.fits,25223040,90220722,EXCLUSIVE_ACCESS,1
90106758,JWST,image,jw02221001001_05101_00005_nrcalong,exposure (L1b): Uncalibrated 4D exposure data,S,mast:JWST/product/jw02221001001_05101_00005_nrcalong_uncal.fits,SCIENCE,--,UNCAL,--,CALJWST,--,2221,jw02221001001_05101_00005_nrcalong_uncal.fits,25223040,90220722,EXCLUSIVE_ACCESS,1
